In [ ]:
#Here is where we import the libraries we will need for this assignment.
import math
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd
import random
from kneed import KneeLocator

random.seed('spearhead')

#To reduce memory usage, we enable copy-on-write mode to avoid making unnecessary copies of the dataframe.
pd.options.mode.copy_on_write = True

In [ ]:
#Now, we load the dataset in a dataframe named "data_table".
data_table = pd.read_csv('Mall_Customers.csv')

#Here, we just display the metadata (column header, missing values, data type) of the dataset:
data_table.info()

In [ ]:
#We remove data with null values and adjust the index of the dataset.
adjusted_dataset = data_table.dropna()
adjusted_dataset = adjusted_dataset.reset_index(drop=True)

#Here, we just display the metadata (column header, missing values, data type) of the dataset:
adjusted_dataset.info()

#We see that there are no data with missing values in the dataset.

In [ ]:
#Here, we select these three features to use for K Means Clustering.
feature1='age'
feature2='annual_income'
feature3='spending_score'

#Then, we create a dataframe containing only these three features (omitting gender).
k_mean_df = adjusted_dataset[[feature1, feature2, feature3]]

In [ ]:
#Here, we visualize the original state of the dataset with a scatter plot having three features.
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(k_mean_df[feature1], k_mean_df[feature2], k_mean_df[feature3], alpha=0.6)
ax.set_xlabel(feature1)
ax.set_ylabel(feature2)
ax.set_zlabel(feature3)
plt.show()

In [ ]:
#Here is the function to plot centroids in random positions (as an initial guess).
def randompoint(feature):
  min = k_mean_df[feature].min()
  max = k_mean_df[feature].max()
  return random.randint(min.astype(np.int64), max.astype(np.int64))

In [ ]:
#Here is a loop for our k means clustering algorithm
wcss = np.array([])

first_centroid = 1
last_centroid = 8

for k in range(first_centroid, last_centroid+1):
    print('\n \nNumber of centroids = ', k)
    number_centroids = k
    centroids = np.array([[randompoint(feature1), randompoint(feature2), randompoint(feature3)] for _ in range(number_centroids)])

    before_wcss = 0

    itr = 1
    while (True):
        print('\nIteration Counter = ', itr)

        #This is how we compute for the distances of the datapoints to the centroids using the Euclidean distance formula where we take the positive values (that's why the values are squared)
        def compute_distances(df, centroids):
            distances = []
            for x in range(df.shape[0]):
                point = [df.iloc[x][feature1], df.iloc[x][feature2], df.iloc[x][feature3]]
                point_distances = []
                for centroid in centroids:
                    distance = ((point[0] - centroid[0]) ** 2 + (point[1] - centroid[1]) ** 2 + (point[2] - centroid[2]) ** 2) ** 0.5
                    point_distances.append(distance)
                distances.append(point_distances)
            return np.array(distances)

        distances_centroids = compute_distances(k_mean_df, centroids)

        #Then, we assign the datapoint to the nearest centroid based from its distance values towards the centroids.
        # - np.argmin(x) finds the index of the nearest centroid for each point.
        # - np.min(x) retrieves the corresponding minimum distance value.
        index_centroids = np.array([[np.argmin(x), np.min(x)] for x in distances_centroids])

        #We store the assignment of the clusters for each datapoint.
        k_mean_df['nearest_centroids'] = index_centroids[:, 0]

        #Then, we store the distances for computing WCSS (Within-Cluster Sum of Squares) later
        current_wcss = index_centroids[:, 1].sum()
        print('WCSS in this iteration = ', current_wcss)

        #For Visualization
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(k_mean_df[feature1], k_mean_df[feature2], k_mean_df[feature3], c=k_mean_df['nearest_centroids'], cmap='viridis', alpha=0.5)
        centroid_x = centroids[:, 0]
        centroid_y =centroids[:, 1]
        centroid_z =centroids[:, 2]
        ax.scatter(centroid_x, centroid_y, centroid_z, c=range(len(centroids)), cmap='viridis', marker='*', s=100, label='Centroids', alpha=1.0)
        ax.set_xlabel(feature1)
        ax.set_ylabel(feature2)
        ax.set_zlabel(feature3)
        plt.show()

        #New centroids are plotted, by again getting the average or mean
        new_centroids = k_mean_df.groupby(by=['nearest_centroids']).mean()
        centroids = np.array([[new_centroids.iloc[i][feature1], new_centroids.iloc[i][feature2], new_centroids.iloc[i][feature3]] for i in range(len(new_centroids))])

        #Go to next iteration
        itr+=1

        #Stop iteration if the values of WCSS has not changed from the previous iteration
        #If WCSS remains the same, it means the centroids have stabilized, so we may now stop looping.
        if (before_wcss == current_wcss):
            print('\nWCSS in this iteration = ', current_wcss)
            wcss = np.append(wcss, current_wcss)
            break
        before_wcss = current_wcss

        #Here is where we print cluster assignments
        print('Cluster Assignment after the iteration: \n', k_mean_df.groupby(by=['nearest_centroids']).count())
    
    print('Final WCSS for this k Cluster = ', before_wcss)

    print('\nFinal Cluster Assignment from the Converged K Means: ', k_mean_df.groupby(by=['nearest_centroids']).count())

print('\nFinal WCSS for all k Clusters tested: ', wcss)

#Plot the Elbow Method Graph
fig, ax = plt.subplots()

ax.xaxis.set_major_locator(ticker.MultipleLocator(1))

len_wcss = [i+1 for i in range(len(wcss))]

print(wcss)

ax.plot(len_wcss, wcss, 'o-', linewidth=2)

plt.show()
